In [1]:
import os
import pickle
import csv
import numpy as np

from sklearn.linear_model import OrthogonalMatchingPursuit, OrthogonalMatchingPursuitCV
from sklearn.pipeline import make_pipeline

#### Функция чтения набора данных

In [2]:
def get_river_dataset(fname, pr_list=None, y_name='H_max'):
    pr_arr = []
    y_arr = []
    with open(fname, newline='') as f:
        reader = csv.DictReader(f, delimiter=';')
        for row in reader:
            pr_arr_row = []
            for pr in pr_list:
                pr_arr_row.append(row[pr])

            pr_arr.append(pr_arr_row)
            y_arr.append(row[y_name])
    X = np.asarray(pr_arr, dtype=np.float64)
    y = np.asarray(y_arr, dtype=np.float64)
    return X, y

#### Функция формирования тестового набора данных с подстановкой нормированных значений

In [3]:
def test_norm(x, pr_list, norms):
    x_norm = np.copy(x)
    for col, pr in enumerate(pr_list):
        if pr in norms:
            x_norm[:, col:col+1] = norms[pr]
    return x_norm

### Функция прогнозирования

In [4]:
def forecast(year, norms=False):

    predict_data_dir = f'data/{year}/Predict'
    
    models_dir = f'results/Models/{year}'

    # Получить список файлов .ipnb из results\Models\<year>
    file_list = tuple(filter(lambda fn: '.pickle' in fn, os.listdir(models_dir)))
    # print(file_list)

    # Сделать множество из кортежей (Название-датасета, группа), преобразовать в список, отсортировать
    dataset_group = set()
    for fn in file_list:
        dataset, year, group, model = fn.split('_')
        dataset_group |= {(dataset, group.split('гр')[1])}
    ds_list = sorted(dataset_group)
    # print(ds_list)

    # Для каждого элемента списка создать result_list[one_model_raw: dict]
    
    for ds, group in ds_list:
        result_list = []
        ds_models = filter(lambda fn: ds in fn and group in fn, file_list)
        for file_name in ds_models:
            with open(f'results/Models/{year}/{file_name}', 'rb') as f:
                model_info = pickle.load(f)
                #model = model_info['Model_full']
                model = model_info['Model_train']
                
                # Прочитать навые признаки (предикторы) для прогноза
                X_new, y = get_river_dataset(f'{predict_data_dir}/{ds}.csv', pr_list=model_info['Predictors_list'])

                # Подстановка нормированных значений для целей тестирования
                if norms:
                    # Подстановка норм в исходный набор признаков
                    X_new = test_norm(X_new, model_info['Predictors_list'], model_info['Norms_data'])

                print(model_info['Dataset_name'])
                print(model_info['Predictors_list'])
                print(model_info['Norms_data'])
                print(model_info['Method'])
                print("y")
                print(y)
                print(f'R2={model_info["R2"]}, R2_t={model_info["R2_t"]}')
                
                y_predicted = np.ravel(model.predict(X_new))
                print("y_predicted")
                print(y_predicted)
                print('X_new')
                print(X_new)
                print(model)
                model_info['Prediction'] = y_predicted[-1]
                result_list.append(model_info)
                # Сортировка результатов по каждому датасету
                result_list.sort(key=lambda row: (row['Criterion'], -row['Correlation'], -row['Pm']))
                print('------------------------------------------------------------------------------')
    
    

In [5]:
forecast(2024, norms=True)

Вилия-Стешицы
['S_2802', 'Smax', 'H_2802', 'X', 'X1', 'X2', 'X3', 'Xs', 'L_max', 'L_2802', 'Q12', 'Q01', 'Q02', 'Y_sum']
{'S_max': 67.0, 'X': 112.0, 'X1': 40.0, 'X2': 33.0, 'L_max': 60.0}
OMP7
y
[281. 292. 275. 291. 168. 195. 166. 198. 286. 277. 207. 292. 216. 198.
 296. 220. 279. 236. 227. 233. 321. 242. 235. 281. 200. 267. 266. 200.
 212. 281.]
R2=0.6502198131701733, R2_t=0.6345423136092758
y_predicted
[284.75059953 286.42450556 300.20779535 249.0291045  195.38656896
 247.65200634 196.12406088 203.88921543 253.78910693 287.81784687
 198.22886535 265.99178725 214.07408662 231.34906537 274.73435546
 232.18888719 270.75068568 246.47499571 196.90551933 238.70676076
 281.04694469 270.52599084 256.6336171  266.2423062  227.30259481
 244.40957308 256.94338336 208.87500488 219.00468318 233.03797842]
X_new
[[ 93.    99.   139.   112.    40.    33.   184.2  146.1   60.    78.
    6.2    4.66   4.5   32.5 ]
 [ 94.    96.   143.   112.    40.    33.   244.2  142.2   60.    78.
    7.29   4.4    